<a href="https://colab.research.google.com/github/minhbao1705/pytorch_tutorial/blob/main/feedforward_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

# Devide configuration

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters

In [3]:
input_size = 28*28
hidden_size = 500
num_classes = 10
num_epochs = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset

In [4]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

--2023-06-08 08:42:15--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2023-06-08 08:42:15--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [     <=>            ]  33.20M  6.07MB/s    in 15s     

2023-06-08 08:42:30 (2.23 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

In [6]:
train_dataset = torchvision.datasets.MNIST(root='./',
                                          train=True,
                                          transform=transforms.ToTensor(),
                                          download=True)

test_dataset = torchvision.datasets.MNIST(root='./',
                                           train=False,
                                           transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# Fully connected neural network with one hidden layer

In [7]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)

    return out

# Create model

In [8]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train the model

In [10]:
total_step = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # Move tensor to the configured device
    images = images.reshape(-1, input_size).to(device)
    labels = labels.to(device)

    # forwaed pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward and optimizer
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print('Epoch [{}/{}], Step[{}/{}], Loss: {}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step[100/600], Loss: 2.2903966903686523
Epoch [1/10], Step[200/600], Loss: 2.264683723449707
Epoch [1/10], Step[300/600], Loss: 2.2350075244903564
Epoch [1/10], Step[400/600], Loss: 2.2164764404296875
Epoch [1/10], Step[500/600], Loss: 2.204686164855957
Epoch [1/10], Step[600/600], Loss: 2.155332088470459
Epoch [2/10], Step[100/600], Loss: 2.129190444946289
Epoch [2/10], Step[200/600], Loss: 2.12100887298584
Epoch [2/10], Step[300/600], Loss: 2.1112585067749023
Epoch [2/10], Step[400/600], Loss: 2.0631842613220215
Epoch [2/10], Step[500/600], Loss: 2.040327310562134
Epoch [2/10], Step[600/600], Loss: 1.997241497039795
Epoch [3/10], Step[100/600], Loss: 2.004701852798462
Epoch [3/10], Step[200/600], Loss: 1.9650310277938843
Epoch [3/10], Step[300/600], Loss: 1.9024335145950317
Epoch [3/10], Step[400/600], Loss: 1.8973864316940308
Epoch [3/10], Step[500/600], Loss: 1.8610594272613525
Epoch [3/10], Step[600/600], Loss: 1.8097773790359497
Epoch [4/10], Step[100/600], Loss: 1.

# Test model

In [11]:
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.reshape(-1, input_size).to(device)
    labels = labels.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Accuracy: {}'.format(100 * correct / total))

Accuracy: 84.58


# Save model

In [12]:
torch.save(model.state_dict(), 'NN_model.ckpt')